### Plano de Avalicação Dell Academy Trilha de Machine Learning

In [1]:
conda install -c conda-forge wordcloud

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

**Atualizando o pandas profiling**

In [2]:
import sys

!"{sys.executable}" -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\leidiana.freitas\\Anaconda3\\Lib\\site-packages\\~arkupsafe\\_speedups.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [11]:
# Standard Library Imports
from pathlib import Path

# Installed packages
import pandas as pd
from ipywidgets import widgets

# Our package
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [20]:
df_test = pd.read_csv("https://raw.githubusercontent.com/izafreitas/machine_learning_dellead_mentoring/main/plano_avalicao_lead/datasets/test.csv?token=ABRCHWCQ7OWJSXO4JYLDTPLBNW5Q6")
df_test.head()

,text,sentiment
0,"My daughter liked it but I was aghast, that a ...",neg
1,I... No words. No words can describe this. I w...,neg
2,this film is basically a poor take on the old ...,neg
3,"This is a terrible movie, and I'm not even sur...",neg
4,First of all this movie is a piece of reality ...,pos


In [ ]:
df_train - pd.read_csv("")